In [139]:
import pandas as pd
import numpy as np
import os
import time
start_time = time.time()

In [140]:
#função de diminuição de dimensionalidade de listas
def flatten_list(_2d_list):
    flat_list = []
    for element in _2d_list:
        if type(element) is list:
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [141]:
dir_path = os.listdir(r'dados_covid')
names_f = [name.split('.')[0] for name  in dir_path] 

In [142]:
# carrega os dados em um dicionário
paises = ['Mexico', 'Argentina', 'Ecuador', 'Chile', 'Spain']
categories = {}
for path,names in zip(dir_path,names_f):
    if path == 'full_grouped.csv':
        categories[names] = pd.read_csv(f'dados_covid\{path}')
        categories[names] = categories[names][categories[names]['Country/Region'].isin(paises)]
    else:
        categories[names] = pd.read_excel(f'dados_covid\{path}')

In [143]:
#Cria as colunas target
columns = ['date','population', 'diabetes_prevalence', 'median_age', 'total_vaccinations', 'hosp_patients','people_fully_vaccinated', 'total_vaccinations_per_hundred','people_vaccinated_per_hundred','people_fully_vaccinated_per_hundred','population_density', 'cardiovasc_death_rate','female_smokers', 'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index']

In [145]:
# Criação dos dados adicionais
new_data = {}
columns = []
columns.append('Country Name')
columns.append([f'{i}' for i in range(2000, 2021)])
columns.append('Type')
columns = flatten_list(columns)
paises_pib = pd.DataFrame({}, columns = columns)
for i in paises:
    country_list = []
    columns.remove('Type')
    for j in categories:
        if j != 'full_grouped':
            data = categories[j].loc[categories[j]['Country Name'] == f'{i}', columns]
            data.insert(0, 'Type', j)
            country_list.append(data)
        else:
            continue
    columns.append('Type')
    country_list = flatten_list(country_list)
    country_temp = pd.DataFrame({}, columns = columns)
    for x in range(len(country_list)):
        list_data = pd.DataFrame(country_list[x], columns=columns)
        country_temp = pd.concat([list_data,country_temp])
    paises_pib = pd.concat([paises_pib,country_temp])
paises_pib.reset_index(drop=True,inplace=True)
for i in paises:
    new_data[i] = paises_pib.loc[paises_pib['Country Name'] == i , :].drop('Country Name', axis = 1).T.rename(columns = {i:k for i,k in zip(paises_pib.T.columns,paises_pib['Type'])}).drop('Type')
    new_data[i].insert(0, 'country', i)
    new_data[i].reset_index().rename(columns={'index': 'years'})
add_info_data = pd.concat([new_data[i] for i in new_data.keys()])
add_info_data.to_excel('add_info.xlsx')

In [166]:
df0 = pd.read_csv(r"E:\Blue\Bootcamp 2\bootcamp_covid\data\raw\twitter_bronze\AR_raw_data.csv")
df0['Country'] = 'Argentina'
df1 = pd.read_csv(r"E:\Blue\Bootcamp 2\bootcamp_covid\data\raw\twitter_bronze\CL_raw_data.csv")
df1['Country'] = 'Chile'
df2 = pd.read_csv(r"E:\Blue\Bootcamp 2\bootcamp_covid\data\raw\twitter_bronze\MX_raw_data.csv")
df2['Country'] = 'Mexico'
df3 = pd.read_csv(r"E:\Blue\Bootcamp 2\bootcamp_covid\data\raw\twitter_bronze\EQ_raw_data.csv")
df3['Country'] = 'Ecuador'
df4 = pd.read_csv(r"E:\Blue\Bootcamp 2\bootcamp_covid\data\raw\twitter_bronze\ES_raw_data.csv")
df4['Country'] = 'Spain'

In [168]:
all = pd.concat([df0,df1,df2,df3,df4])

In [170]:
all.to_excel('twitter_data.xlsx')

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))